# Titanic Tutorial
### Solving Date: Jan 5th, 2024
#### Written by: [Jehwan Kim](https://github.com/kreimben)
#### Followed Materials:
* [https://kaggle-kr.tistory.com/17](https://kaggle-kr.tistory.com/17)
* [https://kaggle-kr.tistory.com/18](https://kaggle-kr.tistory.com/18)

1. 데이터셋 확인 - 대부분의 캐글 데이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니다. 여러 시각화 툴을 사용하여 insight를 얻습니다.
3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 있도록 feature 들을 engineering 합니다. one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
4. model 만들기 - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn 을 사용하면 수많은 알고리즘을 일관된 문법으로 사용할 수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다.
5. 모델 학습 및 예측 - trainset 을 가지고 모델을 학습시킨 후, testset 을 가지고 prediction 합니다.
6. 모델 평가 - 예측 성능이 원하는 수준인지 판단합니다. 풀려는 문제에 따라 모델을 평가하는 방식도 달라집니다. 학습된 모델이 어떤 것을 학습하였는 지 확인해봅니다.

In [ ]:
import pandas as pd
import missingno as msno
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# 1. Dataset Check

In [ ]:
# Pclass   | class of tickets (1st = 1, etc...)
# SibSp    | number of family (sibling and spouse)
# Parch    | number of family (children and parents)
# Ticket   | ticket serial number
# Cabin    | seat serial number
# Embarked | which port the passenger boarded

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.head() # on tutorial variable.

In [ ]:
df_test.head()

In [ ]:
for col in df_train.columns:
    df_train[ col ].info()
    print('==========================')

> 우리가 다루는 문제에서 feature는 Pclass, Age, SibSp, Parch, Fare 이며, 예측하려는 target label 은 **Survived** 입니다.

In [ ]:
df_train.describe() # desc about fields.

In [ ]:
for col in df_train.columns:
    print(f'{col=:>12}    Null %: {100 * (df_train[col].isnull().sum() / df_train[col].shape[0]):.2f}%')
    print(f'\t\t    NaN %: {100 * (df_train[col].isna().sum() / df_train[col].shape[0]):.2f}%')
    print()

In [ ]:
for col in df_test.columns:
    print(f'{col=:>12}    Null %: {100 * (df_test[col].isnull().sum() / df_test[col].shape[0]):.2f}%')
    print(f'\t\t    NaN %: {100 * (df_test[col].isna().sum() / df_test[col].shape[0]):.2f}%')
    print()

For the pure curiosity between `isna` and `isnull`, Found the answer.

`isna` and `isnull` are **identical** as well.

But there are some recommendations **about using `isna` over `isnull`**. ([stackoverflow](https://stackoverflow.com/questions/52086574/pandas-isna-and-isnull-what-is-the-difference))

To check `null` data more visually, using `missingno` library.

In [ ]:
msno.matrix(
    df=df_train,#.iloc[:, :], it's same!
    figsize=(4, 4),
    color=(0.8, 0.5, 0.2)
)

In [ ]:
msno.bar(
    df=df_train,
    figsize=(4, 4),
    color=(0.8, 0.5, 0.2)
)

## Checking Target Label
> target label이 어떤 distribution 을 가지고 있는 지 확인해봐야 합니다.
> 지금 같은 binary classification 문제의 경우에서, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라 질 수 있습니다.

In [ ]:
# To get more definitive data, explicitly by words,
# I refined the data frame.
df_train_def = df_train.copy()
df_train_def['Survived'] = df_train['Survived'].apply(lambda x: 'alive' if x == 1 else 'died')
df_train_def.head(2)
# BUT, THERE ARE PROS AND CONS OF THAT WAY.
# FOR EXAMPLE, I CANNOT GET THE AVERAGE VALUES (RATIO) ON 'HOW MANY PEOPLE SURVIVED?'.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# left chart
df_train_def['Survived'].value_counts().plot\
    .pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')

# right chart
sns.countplot(x='Survived', data=df_train_def, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt

> 이제 본격적으로 데이터 분석을 해보겠습니다. 데이터는 매우 많습니다. 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요합니다.

> 시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있습니다. 특정 목적에 맞는 소스코드를 정리해두어 필요할 때마다 참고하면 편합니다.

### `Pclass` visualisation

In [ ]:
# Either `ordinal` and `category` itself.
df_train['Pclass'].info()

To check how many people survived by `Pclass`,

In [ ]:
# COUNT
df_train[['Pclass', 'Survived']]\
    .groupby(['Pclass'], as_index=True).count()

In [ ]:
# SUM
df_train[['Pclass', 'Survived']]\
    .groupby(['Pclass'], as_index=True).sum()

In [ ]:
pd.crosstab(
    df_train['Pclass'], df_train_def['Survived'], # for the readability...
    margins=True
).style.background_gradient(cmap='summer_r')

To get a mean value by a class (`Pclass`)...

In [ ]:
df_train[['Pclass', 'Survived']]\
    .groupby(['Pclass'], as_index=True)\
    .mean()\
    .sort_values(by='Survived', ascending=False)\
    .plot.bar()

In [ ]:
y_position = 1.02

f, ax = plt.subplots(1, 2, figsize=(18, 8))

# left chart
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_title('Number of passengers by a class', y=y_position)
ax[0].set_ylabel('Count')

# right chart
sns.countplot(x='Pclass', hue='Survived', data=df_train_def, ax=ax[1])
ax[1].set_title('Pclass: Survived vs Dead', y=y_position)

plt

### `sex` visualisation

It's gonna same as `Pclass`...

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# left chart
df_train[['Sex', 'Survived']]\
    .groupby(['Sex'], as_index=True)\
    .mean()\
    .plot.bar(ax=ax[0])
ax[0].set_title('Surviving Ratio by Sex')

# right chart
sns.countplot(x='Sex', hue='Survived', data=df_train_def, ax=ax[1])
ax[1].set_title('Sex: Survived vs Dead')

plt

In [ ]:
df_train[['Sex', 'Survived']]\
    .groupby(['Sex'], as_index=True)\
    .mean().sort_values(by='Survived', ascending=True)

In [ ]:
pd.crosstab(
    df_train['Sex'], df_train['Survived'],
    margins=True
)\
    .style.background_gradient(cmap='summer_r')

### Both Sex and Pclass

> 이번에는 `Sex`, `Pclass` 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 봅시다

> seaborn 의 factorplot 을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있습니다.

In [ ]:
# sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train, size=6, aspect=1.5)
sns.catplot(x='Pclass', y='Survived', hue='Sex', data=df_train, kind='point')

In [ ]:
# sns.factorplot(x='Sex', y='Survived', col='Pclass',
#               data=df_train, satureation=.5,
#                size=9, aspect=1)
sns.catplot(x='Sex', y='Survived', col='Pclass',
            data=df_train, saturation=.5,
            kind='point')

### Age

In [ ]:
print(f'''
The oldest passenger: {df_train['Age'].max():.1f} Y/O
The youngest passenger: {df_train['Age'].min():.1f} Y/O
The average age of passenger: {df_train['Age'].mean():.1f} Y/O
''')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 5))

sns.kdeplot(df_train[ df_train['Survived'] == 1 ]['Age'], ax=ax)
sns.kdeplot(df_train[ df_train['Survived'] == 0 ]['Age'], ax=ax)

plt.legend(['Survived', 'Dead'])

In [ ]:
# Age distribution within class

plt.figure(figsize=(6, 4))

df_train['Age'][ df_train['Pclass'] == 1].plot(kind='kde')
df_train['Age'][ df_train['Pclass'] == 2].plot(kind='kde')
df_train['Age'][ df_train['Pclass'] == 3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within class')
plt.legend(['1st Class', '2nd Class', '3rd Class'])

Class 가 높을 수록 나이 많은 사람의 비중이 커짐.

나이대가 변하면서 생존률이 어떻게 되는 지 보려고 합니다.

나이범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 봅시다.

In [ ]:
ratio = []

for i in range(1, 80):
    target = df_train[ df_train['Age'] < i]['Survived']
    ratio.append(
        target.sum() / target.count()
    )

plt.figure(figsize=(5, 5))
plt.plot(ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age')

### Pclass, Sex, Age

> 지금까지 본, `Sex`, `Pclass`, `Age`, `Survived` 모두에 대해서 보고싶습니다. 이를 쉽게 그려주는 것이 seaborn 의 violinplot 입니다.

> x 축은 우리가 나눠서 보고싶어하는 case(여기선 `Pclass`, `Sex`) 를 나타내고, y 축은 보고 싶어하는 distribution(Age) 입니다.

> 한번 그려보겠습니다

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

# left chart
sns.violinplot(x='Pclass', y='Age', hue='Survived', data=df_train, density_norm='count', split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))

# right chart
sns.violinplot(x='Sex', y='Age', hue='Survived', data=df_train, density_norm='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))


1. 생존만 봤을 때, 모든 클래스에서 **나이가 어릴 수록 생존을 많이 한것을 볼 수 있습니다.**
2. 오른쪽 그림에서 보면, **명확히 여자가 생존을 많이 한것을 볼 수 있습니다.**
3. **여성과 아이를 먼저 챙긴 것을 볼 수 있습니다.**

### Embarked

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))

df_train[['Embarked', 'Survived']]\
    .groupby(['Embarked'], as_index=True)\
    .mean().sort_values(by='Survived', ascending=False)\
    .plot.bar(ax=ax)

In [ ]:
f, ax = plt.subplots(2, 2, figsize=(20, 15))

sns.countplot(x='Embarked', data=df_train, ax=ax[0, 0])
ax[0, 0].set_title('(1) Number of Passengers Boarded')

sns.countplot(x='Embarked', hue='Sex', data=df_train, ax=ax[0, 1])
ax[0, 1].set_title('(2) Male-Female Split for Embarked')

sns.countplot(x='Embarked', data=df_train, hue='Survived', ax=ax[1, 0])
ax[1, 0].set_title('(3) Embarked vs Survived')

sns.countplot(x='Embarked', data=df_train, hue='Pclass', ax=ax[1, 1])
ax[1, 1].set_title('(4) Embarked vs Pclass')

plt.subplots_adjust(wspace=0.2, hspace=0.5)

### Family - SibSp (형제, 배우자) + Parch (부모, 자녀)

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 나 자신
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 # 나 자신

In [ ]:
# Relativeness of number of family

f, ax = plt.subplots(1, 3, figsize=(40, 10))

sns.countplot(x='FamilySize', data=df_train, ax=ax[0])
ax[0].set_title('(1) Number of Passengers Boarded', y=1.02)

sns.countplot(x='FamilySize', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize')

df_train[['Survived', 'FamilySize']].groupby('FamilySize')\
    .mean().sort_values(by='Survived', ascending=False)\
    .plot.bar(ax=ax[2])

가족의 수가 4명인 가정일 수록 생존 확률이 높아짐.
의외로 1인 탑승객은 생존률이 그다지 높지 않은 것을 볼 수 있음...

### Fare

Fare is how much money passengers paid for ticket and it's continuous.
Let's check it's skewness first.

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))

g = sns.distplot(df_train['Fare'], color='b', label=f'Skewness: {df_train["Fare"].skew():.2f}', ax=ax)
g = g.legend(loc='best')

> 보시다시피, distribution이 매우 비대칭인 것을 알 수 있습니다 (high skewness). 만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있습니다. 몇개 없는 outlier 에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있습니다.

1. 과적합의 문제인가?
2. log를 써서 평탄화를 하면 outlier의 영향으로부터 자유로운가? 차라리 튀는 값들을 없애는것이 더 낫지 않은가?

In [ ]:
df_test.loc[ df_test.Fare.isna(), 'Fare' ] = df_test['Fare'].mean() # just substitute nan value as a mean value.

df_train['Fare'] = df_train['Fare'].apply(lambda x: np.log(x) if x > 0 else 0)
df_test['Fare'] = df_test['Fare'].apply(lambda x: np.log(x) if x > 0 else 0)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(8, 8))

g = sns.distplot(df_train.Fare, color='b', label=f'Skewness: {df_train.Fare.skew():.2f}', ax=ax)
g = g.legend(loc='best')

### Cabin
pass it cuz there are so many nan values.

### Ticket

In [ ]:
df_train.Ticket.value_counts()

> 보시다시피, ticket number 는 매우 다양합니다. 우리는 여기서 어떤 특징을 이끌어내서 생존과 연결시킬 수 있을까요?

> 여러분이 직접 한번 아이디어를 내보세요! 이것이 본격적인 캐글 레이스의 시작점입니다 ^^

...
티켓 넘버로 무언가를 해볼 수 있을까?
일련번호는 특수한 규칙을 이용해 만들어지거나 단순히 누적합 일 수 있다.
ML의 영역에서 serial을 분석하는게 가능할까...

## 3. Feature Engineering

* fill the `na` value.
* performances depend on how to fill the `na` value.
* test set should be adjusted as same as train set be did.

### fill null in `Age` using title

In [ ]:
# how many null data in Age feature?
count = 0
for x in df_train.Age.isna():
    if x: count += 1
print(f'There are {count} empty values in Age feature!')

> Age 에는 null data가 177개나 있습니다. 이를 채울 수 있는 여러 아이디어가 있을 것인데, 여기서 우리는 title + statistics 를 사용해 보겠습니다.

In [ ]:
df_train['Initial'] = df_train.Name.str.extract('([A-Za-z]+)\.') # let's extract the saluations.
df_test['Initial'] = df_test.Name.str.extract('([A-Za-z]+)\.') # let's extract the saluations.

In [ ]:
df_train.head(2)

In [ ]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r')

In [ ]:
df_train['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],
    inplace=True
)

df_test['Initial'].replace(
    ['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
    ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr']
    ,inplace=True
)

In [ ]:
df_train[['Initial', 'Survived']].groupby('Initial').mean()

In [ ]:
df_train.groupby('Initial')['Survived'].mean().plot.bar()

In [ ]:
df_train[['Initial', 'Survived', 'Pclass', 'Age']].groupby('Initial').mean()

In [ ]:
df_initial = df_train[['Initial', 'Age']].groupby('Initial').mean()
df_initial

Now, I will fill `na` using this mean value.

In [ ]:
# populate the main titles EXCEPT 'Other'.
df_initial.index
# df_initial.loc['Master']

In [ ]:
df_bac = df_train.copy() # just for backup.
df_bac

In [ ]:
df_age_mean = df_bac.groupby(['Survived','Initial'])['Age'].transform('mean')

In [ ]:
# df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mr'),'Age'] = 33
# df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mrs'),'Age'] = 36
# df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Master'),'Age'] = 5
# df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Miss'),'Age'] = 22
# df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Other'),'Age'] = 46

# df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mr'),'Age'] = 33
# df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mrs'),'Age'] = 36
# df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Master'),'Age'] = 5
# df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Miss'),'Age'] = 22
# df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Other'),'Age'] = 46

df_bac['Age'] = df_bac['Age'].fillna(df_age_mean)

In [ ]:
df_bac.isna().sum()

In [ ]:
df_bac[ df_bac.Age.isna() ].Initial

In [ ]:
df_train = df_bac
del df_bac, df_initial

In [ ]:
df_test['Age'] = df_train.Age

Filled every nan data in age feature!

### Fill null in Embarked

In [ ]:
# How many people don't have embarked feature?
df_train.Embarked.isna().sum()

In [ ]:
# df_train#['Embarked']
# df_pivot = pd.pivot(df_train, index='Embarked', columns='Age')
# df_pivot
# Embarked 별로 몇개씩 있는지 집계 테이블을 만들고 싶었지만...

# 1.
df_train.groupby('Embarked').count()['Age']

# 2.
df_train.Embarked.value_counts()

In [ ]:
# Fill the value with `S`.
df_train['Embarked'].fillna('S', inplace=True)

## Change Initial, Embarked and Sex (String to Numerical)

In [ ]:
df_train['Initial'] = df_train['Initial'].map({
    'Master': 0,
    'Miss': 1,
    'Mr': 2,
    'Mrs': 3,
    'Other': 4
})

df_test['Initial'] = df_test['Initial'].map({
    'Master': 0,
    'Miss': 1,
    'Mr': 2,
    'Mrs': 3,
    'Other': 4
})

In [ ]:
df_train.Embarked.unique()

In [ ]:
df_train.Embarked.value_counts()

In [ ]:
df_train['Embarked'] = df_train['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
df_test['Embarked'] = df_test['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})

In [ ]:
# test if there are still na values.
df_train.Embarked.isna().sum()
df_test.Embarked.isna().sum()

In [ ]:
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1})

#### Fix every string values to numeric values for calculating relevance.

Draw heapmap using `corr()`.

In [ ]:
data = df_train[['Survived', 'Pclass', 'Sex', 'Fare', 'Embarked', 'FamilySize', 'Initial']]

color_map = plt.cm.RdBu
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(data.astype(float).corr(), linewidths=0.1, vmax=1.0, 
            square=True, cmap=color_map, linecolor='white', annot=True, annot_kws={'size': 16})

Fare과 Pclass의 높은 상관관계를 띄고 있으나 상식적인 내용이라 pass,
그 외에는 성별과 생존여부가 유의미하게 나왔다.
딱히 상관관계가 있는 feature들이 없어보인다.

### One-hot encoding on Initial and Embarked

In [ ]:
# For the model's performance
df_train = pd.get_dummies(df_train, columns=['Initial'], prefix='Initial')
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')

In [ ]:
df_train.head(2)

In [ ]:
# 마찬가지로 Embarked feature도 one-hot encoding을 해준다... (아직 이게 어떻게 왜 성능을 향상 시킨다는건지 모르겠다...)
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

### Drop Columns

In [ ]:
df_train.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId', 'Name',  'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

## Building machine learning model and prediction using the trained model

In [ ]:
# split dataset into train, valid, test set

X_train = df_train.drop('Survived', axis=1).values
target_label = df_train['Survived'].values
X_test = df_test.values

# test -> valid -> test

In [ ]:
X_tr, X_vld, Y_tr, Y_vld = train_test_split(X_train, target_label, test_size=0.2, random_state=990321)
# test_size 0.3 => split the dataset 80% for test and 20% for valid dataset.
# random_state = random seed.

In [ ]:
model = RandomForestClassifier()
model.fit(X_tr, Y_tr)
prediction = model.predict(X_vld)

In [ ]:
f'Predicted survival {100 * metrics.accuracy_score(prediction, Y_vld):.2f}% out of {Y_vld.shape[0]} people'

## Feature Importance

지금 모델은 당연히 학습된 feature들에 영향을 가짐. 그래서 얼마나 그 수치를 가졌는지 구해보는 단계.

In [ ]:
from pandas import Series

feature_importance = model.feature_importances_
series_feat_imp = Series(feature_importance, index=df_test.columns)

In [ ]:
feature_importance

In [ ]:
plt.figure(figsize=(8, 8))
series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt

위에서 봤듯이 나이와 요금이 높은 상관관계를 보였고, initial 2인 'Mr'이 영향을 줬다.
블로그와 그래프가 낮게 나온 이유는 나이대를 분류화 하지 않아서 그러지 않을까 싶다.

## Prediction on Test set

In [ ]:
submission = pd.read_csv('gender.csv')

In [ ]:
submission.head()

In [ ]:
X_test = df_test.values
prediction = model.predict(X_test)
prediction

In [ ]:
submission['Survived'] = prediction

In [ ]:
submission

In [ ]:
submission.to_csv('tistory_blog_submission.csv', index=False)

![image](kaggle_leaderboard_1.png "Kaggle Leaderboard")